In [10]:
sc.install_pypi_package("boto3")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
from pyspark.sql import SparkSession
import boto3
from datetime import datetime

def get_processed_files(table):
    response = table.scan()
    return {item['FileName'] for item in response['Items']}

def update_processed_files(table, file_name):
    response = table.put_item(
        Item={
            'FileName': file_name,
            'ProcessedDate': datetime.now().isoformat()
        }
    )
    return response

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
dynamodb = boto3.resource('dynamodb', region_name="ap-northeast-2")
table = dynamodb.Table('ProcessedFiles')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
processed_files = get_processed_files(table)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred (ResourceNotFoundException) when calling the Scan operation: Requested resource not found
Traceback (most recent call last):
  File "<stdin>", line 6, in get_processed_files
  File "/mnt1/yarn/usercache/livy/appcache/application_1714370915645_0002/container_1714370915645_0002_01_000001/tmp/spark-b88befb0-4c70-41b5-8c45-ab14c5618814/lib/python3.7/site-packages/boto3/resources/factory.py", line 581, in do_action
    response = action(self, *args, **kwargs)
  File "/mnt1/yarn/usercache/livy/appcache/application_1714370915645_0002/container_1714370915645_0002_01_000001/tmp/spark-b88befb0-4c70-41b5-8c45-ab14c5618814/lib/python3.7/site-packages/boto3/resources/action.py", line 88, in __call__
    response = getattr(parent.meta.client, operation_name)(*args, **params)
  File "/mnt1/yarn/usercache/livy/appcache/application_1714370915645_0002/container_1714370915645_0002_01_000001/tmp/spark-b88befb0-4c70-41b5-8c45-ab14c5618814/lib/python3.7/site-packa

In [ ]:
file_list = spark.sparkContext.wholeTextFiles("s3://your-bucket/path/*").keys().collect()

In [1]:
catalog_name = "glue_catalog"
bucket_name = "chiholee-datalake001"
database_name = "ecommerce"

table_name = "orders"
last_update_time = 'order_dt'

source_bucket_prefix = "transaction/initial/raw"
source_path = f"s3://{bucket_name}/{source_bucket_prefix}"
source_table_name = table_name

iceberg_bucket_prefix = "transaction/iceberg/emr"
warehouse_path = f"s3://{bucket_name}/{iceberg_bucket_prefix}"
iceberg_table_name = f"{table_name}_cdc_emr_iceberg"


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1714370915645_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .config(f"spark.sql.catalog.{catalog_name}", "org.apache.iceberg.spark.SparkCatalog") \
    .config(f"spark.sql.catalog.{catalog_name}.warehouse", f"{warehouse_path}") \
    .config(f"spark.sql.catalog.{catalog_name}.catalog-impl", "org.apache.iceberg.aws.glue.GlueCatalog") \
    .config(f"spark.sql.catalog.{catalog_name}.io-impl", "org.apache.iceberg.aws.s3.S3FileIO") \
    .config("spark.sql.extensions","org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
fullDf = spark.read.parquet(f'{source_path}/{database_name}/{source_table_name}/')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
from datetime import datetime
from pyspark.sql.functions import year, month, dayofmonth
from pyspark.sql.functions import concat, col, lit, to_timestamp

current_datetime = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
fullDf = fullDf.withColumn(last_update_time,to_timestamp(col(last_update_time)))
fullDf = (fullDf
      .withColumn('year', year(col(last_update_time)))
      .withColumn('month', month(col(last_update_time)))
      .withColumn('day', dayofmonth(col(last_update_time)))
     )
fullDf = fullDf.withColumn('last_applied_date',to_timestamp(lit(current_datetime)))





VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
fullDf = fullDf.repartition("year", "month", "day").sortWithinPartitions("year", "month", "day")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
fullDf.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------+---------+-----------+-------------------+-----------+----------+----+-----+---+-------------------+
|order_id|promo_id|order_cnt|order_price|           order_dt|customer_id|product_id|year|month|day|  last_applied_date|
+--------+--------+---------+-----------+-------------------+-----------+----------+----+-----+---+-------------------+
|       7| PROMO19|        1|       6000|2024-04-19 10:14:53|         76|        12|2024|    4| 19|2024-04-30 07:49:50|
|      14| PROMO08|        4|      15000|2024-04-19 05:32:45|         90|        18|2024|    4| 19|2024-04-30 07:49:50|
|      29| PROMO13|        1|      45000|2024-04-19 01:08:33|         10|         9|2024|    4| 19|2024-04-30 07:49:50|
|      54| PROMO11|        9|      23000|2024-04-19 07:22:30|         81|         6|2024|    4| 19|2024-04-30 07:49:50|
|      69| PROMO15|       10|      22000|2024-04-19 23:27:57|         27|        17|2024|    4| 19|2024-04-30 07:49:50|
|      90| PROMO11|        1|      36000

In [18]:
fullDf.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- order_id: integer (nullable = true)
 |-- promo_id: string (nullable = true)
 |-- order_cnt: integer (nullable = true)
 |-- order_price: integer (nullable = true)
 |-- order_dt: timestamp (nullable = true)
 |-- customer_id: long (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- last_applied_date: timestamp (nullable = true)

In [22]:
fullDf.createOrReplaceTempView(f"{source_table_name}_initial")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
spark.sql(f"CREATE DATABASE IF NOT EXISTS {catalog_name}.{database_name}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [23]:
spark.sql(f"""CREATE TABLE IF NOT EXISTS {catalog_name}.{database_name}.{iceberg_table_name}
            USING iceberg 
            PARTITIONED BY (year, month, day)
            as (SELECT * from {source_table_name}_initial)""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]